In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import statsmodels
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.discrete.discrete_model import Logit

from scipy import stats
from scipy.stats import skew,norm
from scipy.stats.stats import pearsonr

import seaborn as sns
mpl.rcParams['toolbar'] = 'None'

In [70]:
df = pd.read_excel('/Users/lesson6_mac/Downloads/TeamProject/comb_mini.xlsx')

In [71]:
df['Cause'] = np.where(df['Cause']=='개인질환',1,0)
df.head(10)

,Cause,humi,temp,wind,rain,dew,snow
0,0,50.00,-2.750,3.550,0.0,-9.050,0.0
1,0,69.25,19.125,1.300,0.0,14.750,0.0
2,0,50.00,11.425,2.550,0.0,4.900,0.0
3,0,45.75,24.050,2.500,0.0,14.425,0.0
4,0,28.25,28.800,1.425,0.0,12.000,0.0
5,0,50.50,30.425,1.625,0.0,21.350,0.0
6,0,35.00,25.450,1.300,0.0,10.525,0.0
7,0,60.00,23.925,2.625,0.0,18.875,0.0
8,0,47.00,29.925,1.650,0.0,21.175,0.0
9,0,58.25,32.050,1.350,0.0,24.950,0.0


In [72]:
df_1 = df[df['Cause']==1].sample(n =100)
df_0 = df[df['Cause']==0].sample(n =100)
df = df_1.append(df_0)

In [48]:
from statsmodels.discrete.discrete_model import Logit
model = Logit.from_formula("Cause ~ humi + temp + wind ", df) 
result = model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.679245
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                  Cause   No. Observations:                  199
Model:                          Logit   Df Residuals:                      195
Method:                           MLE   Df Model:                            3
Date:                Wed, 26 Jun 2019   Pseudo R-squ.:                 0.02004
Time:                        15:23:06   Log-Likelihood:                -135.17
converged:                       True   LL-Null:                       -137.93
                                        LLR p-value:                    0.1370
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3768      0.701     -1.964      0.050      -2.751      -0.003
humi           0.0196      0.

In [49]:
test = df[['humi','temp','wind']]
test

,humi,temp,wind
30056,57.50,15.600,1.800
29609,51.50,8.150,2.900
29133,40.25,23.425,1.400
30535,48.00,4.850,0.050
30929,60.00,4.800,0.050
31356,NaN,23.100,2.700
30822,67.00,24.350,0.600
31018,43.00,6.250,2.400
31513,55.50,22.250,3.950
31387,42.50,-2.800,3.500


In [51]:
df.head()

,Cause,humi,temp,wind,rain,dew,snow,predict
30056,1,57.50,15.600,1.80,0.0,7.65,0.0,0.519827
29609,1,51.50,8.150,2.90,0.0,0.90,0.0,0.502469
29133,1,40.25,23.425,1.40,0.0,12.30,0.0,0.442615
30535,1,48.00,4.850,0.05,0.0,-1.20,0.0,0.404525
30929,1,60.00,4.800,0.05,0.0,-1.40,0.0,0.462137


In [66]:
from statsmodels.discrete.discrete_model import Logit
model = Logit.from_formula("Cause ~ humi + temp + wind ", df) 
result = model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.666602
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                  Cause   No. Observations:                  200
Model:                          Logit   Df Residuals:                      196
Method:                           MLE   Df Model:                            3
Date:                Wed, 26 Jun 2019   Pseudo R-squ.:                 0.03830
Time:                        15:32:20   Log-Likelihood:                -133.32
converged:                       True   LL-Null:                       -138.63
                                        LLR p-value:                   0.01398
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.5777      0.701      2.250      0.024       0.203       2.952
humi          -0.0168      0.

In [67]:
df_1 = df[df['Cause']==1].sample(n =100)
df_0 = df[df['Cause']==0].sample(n =100)
df_test = df_1.append(df_0)

In [68]:
df['predict1'] = result.predict(df_test)

In [69]:
result.pred_table()

array([[54., 46.],
       [43., 57.]])

In [73]:
from statsmodels.discrete.discrete_model import Logit
model = Logit.from_formula("Cause ~ humi + temp + wind ", df) 
result = model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.638140
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                  Cause   No. Observations:                  200
Model:                          Logit   Df Residuals:                      196
Method:                           MLE   Df Model:                            3
Date:                Wed, 26 Jun 2019   Pseudo R-squ.:                 0.07936
Time:                        15:34:27   Log-Likelihood:                -127.63
converged:                       True   LL-Null:                       -138.63
                                        LLR p-value:                 6.514e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.3210      0.687      1.922      0.055      -0.026       2.668
humi          -0.0070      0.

In [131]:
df = pd.read_excel('/Users/lesson6_mac/Downloads/TeamProject/comb_mini.xlsx')
df['Cause'] = np.where(df['Cause']=='개인질환',1,0)
df_1 = df[df['Cause']==1].sample(n =10000,replace = True)
df_0 = df[df['Cause']==0].sample(n =10000, replace = True)
df = df_1.append(df_0)
from statsmodels.discrete.discrete_model import Logit
model = Logit.from_formula("Cause ~ humi  + temp + dew + rain + snow", df) 
result = model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.688793
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                  Cause   No. Observations:                19988
Model:                          Logit   Df Residuals:                    19982
Method:                           MLE   Df Model:                            5
Date:                Wed, 26 Jun 2019   Pseudo R-squ.:                0.006282
Time:                        15:49:59   Log-Likelihood:                -13768.
converged:                       True   LL-Null:                       -13855.
                                        LLR p-value:                 9.940e-36
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8014      0.187     -9.643      0.000      -2.168      -1.435
humi           0.0201      0.

In [132]:
result.pred_table()

array([[5519., 4476.],
       [4684., 5309.]])